<a href="https://colab.research.google.com/github/fovi-llc/unsloth-notebooks/blob/main/nb/Mistral_Small_22B_GRPO.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

To run this, press "*Runtime*" and press "*Run all*" on a **free** Tesla T4 Google Colab instance!
<div class="align-center">
<a href="https://unsloth.ai/"><img src="https://github.com/unslothai/unsloth/raw/main/images/unsloth%20new%20logo.png" width="115"></a>
<a href="https://discord.gg/unsloth"><img src="https://github.com/unslothai/unsloth/raw/main/images/Discord button.png" width="145"></a>
<a href="https://docs.unsloth.ai/"><img src="https://github.com/unslothai/unsloth/blob/main/images/documentation%20green%20button.png?raw=true" width="125"></a></a> Join Discord if you need help + ⭐ <i>Star us on <a href="https://github.com/unslothai/unsloth">Github</a> </i> ⭐
</div>

To install Unsloth on your own computer, follow the installation instructions on our Github page [here](https://docs.unsloth.ai/get-started/installing-+-updating).

You will learn how to do [data prep](#Data), how to [train](#Train), how to [run the model](#Inference), & [how to save it](#Save)


### News

**Read our [blog post](https://unsloth.ai/blog/r1-reasoning) for guidance on how to train reasoning models.**

Visit our docs for all our [model uploads](https://docs.unsloth.ai/get-started/all-our-models) and [notebooks](https://docs.unsloth.ai/get-started/unsloth-notebooks).


### Installation

In [1]:
# %%capture
# Skip restarting message in Colab
import sys; modules = list(sys.modules.keys())
for x in modules: sys.modules.pop(x) if "PIL" in x or "google" in x else None

!pip install "unsloth==2025.2.9" vllm
!pip install --upgrade pillow
# If you are running this notebook on local, you need to install `diffusers` too
# !pip install diffusers
# Temporarily install a specific TRL nightly version
!pip install git+https://github.com/huggingface/trl.git@e95f9fb74a3c3647b86f251b7e230ec51c64b72b

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.5/57.5 kB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 185.6/185.6 kB 8.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 264.3/264.3 MB 8.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 96.5/96.5 kB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.6/71.6 kB 6.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 111.0/111.0 kB 11.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 87.6/87.6 kB 9.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.7/16.7 MB 94.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 104.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 87.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 57.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66

Mount GDrive for persistence

In [2]:
from google.colab import drive
drive.mount('/content/drive')

%cd /content/drive/MyDrive/Data/Unsloth

import datetime
now = datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
run_dir = f"mistral-grpo_{now}"
!mkdir {run_dir}
%cd {run_dir}

Mounted at /content/drive
/content/drive/MyDrive/Data/Unsloth
/content/drive/MyDrive/Data/Unsloth/mistral-grpo_20250215-035142


In [14]:
!pwd
hf_hub_path = f"jimwhite/{run_dir}"
hf_hub_path

/content/drive/MyDrive/Data/Unsloth/mistral-grpo_20250215-035142


'jimwhite/mistral-grpo_20250215-035142'

### Unsloth

Use `PatchFastRL` before all functions to patch GRPO and other RL algorithms!

In [4]:
from unsloth import FastLanguageModel, PatchFastRL
PatchFastRL("GRPO", FastLanguageModel)

Unsloth: Patching Xformers to fix some performance issues.
🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!
INFO 02-15 03:53:00 __init__.py:190] Automatically detected platform cuda.


Load up `Mistral-Small-24B-Instruct`, and set parameters

In [5]:
from unsloth import is_bfloat16_supported
import torch
from trl import SFTTrainer
from transformers import TrainingArguments
import re
from datasets import load_dataset, Dataset
max_seq_length = 8096 # Supports RoPE Scaling interally, so choose any!

lora_rank = 64
lora_alpha = 64

# 4bit pre quantized models we support for 4x faster downloading + no OOMs.
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/Mistral-Small-24B-Instruct-2501",
    max_seq_length = max_seq_length,
    dtype = None,
    fast_inference = True,
    max_lora_rank = lora_rank,
    # gpu_memory_utilization = 0.5,
    gpu_memory_utilization = 0.8,
    load_in_4bit = True,
    # load_in_8bit = True, there is only unsloth/Mistral-Small-24B-Instruct-2501-unsloth-bnb-4bit no -8bit
)

# Do model patching and add fast LoRA weights
model = FastLanguageModel.get_peft_model(
    model,
    r = lora_rank,
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj",],
    lora_alpha = lora_alpha,
    lora_dropout = 0,
    # Supports any, but = 0 is optimized
    bias = "none",    # Supports any, but = "none" is optimized
    # [NEW] "unsloth" uses 30% less VRAM, fits 2x larger batch sizes!
    use_gradient_checkpointing = "unsloth", # True or "unsloth" for very long context
    random_state = 3407,
    use_rslora = False,  # We support rank stabilized LoRA
    loftq_config = None, # And LoftQ
)

==((====))==  Unsloth 2025.2.9: Fast Mistral patching. Transformers: 4.48.3.
   \\   /|    GPU: NVIDIA A100-SXM4-40GB. Max memory: 39.557 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.5.1+cu124. CUDA: 8.0. CUDA Toolkit: 12.4. Triton: 3.1.0
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.28.post3. FA2 = False]
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!
Unsloth: vLLM loading unsloth/mistral-small-24b-instruct-2501-unsloth-bnb-4bit with actual GPU utilization = 79.16%
Unsloth: Your GPU has CUDA compute capability 8.0 with VRAM = 39.56 GB.
Unsloth: Using conservativeness = 1.0. Chunked prefill tokens = 8096. Num Sequences = 256.
Unsloth: vLLM's KV Cache can use up to 14.87 GB. Also swap space = 6 GB.
INFO 02-15 03:53:50 config.py:542] This model supports multiple tasks: {'classify', 'embed', 'generate', 'reward', 'score'}. Defaulting to 'generate'.
Unsloth: vLLM Bitsandbytes co

tokenizer_config.json:   0%|          | 0.00/200k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.1M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/21.4k [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/205 [00:00<?, ?B/s]

...

INFO 02-15 03:55:57 model_runner.py:1562] Graph capturing finished in 51 secs, took 1.22 GiB
INFO 02-15 03:55:57 llm_engine.py:431] init engine (profile, create kv cache, warmup model) took 67.77 seconds


tokenizer_config.json:   0%|          | 0.00/200k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/21.4k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.1M [00:00<?, ?B/s]

Unsloth 2025.2.9 patched 40 layers with 40 QKV layers, 40 O layers and 40 MLP layers.


### Data Prep
<a name="Data"></a>

We directly leverage [@willccbb](https://gist.github.com/willccbb/4676755236bb08cab5f4e54a0475d6fb) for data prep and all reward functions. You are free to create your own!

In [6]:
# Load and prep dataset
SYSTEM_PROMPT = """
Respond in the following format:
<reasoning>
...
</reasoning>
<answer>
...
</answer>
"""

XML_COT_FORMAT = """\
<reasoning>
{reasoning}
</reasoning>
<answer>
{answer}
</answer>
"""

def extract_xml_answer(text: str) -> str:
    answer = text.split("<answer>")[-1]
    answer = answer.split("</answer>")[0]
    return answer.strip()

def extract_hash_answer(text: str) -> str | None:
    if "####" not in text:
        return None
    return text.split("####")[1].strip()

# uncomment middle messages for 1-shot prompting
def get_gsm8k_questions(split = "train") -> Dataset:
    data = load_dataset('openai/gsm8k', 'main')[split] # type: ignore
    data = data.map(lambda x: { # type: ignore
        'prompt': [
            {'role': 'system', 'content': SYSTEM_PROMPT},
            {'role': 'user', 'content': x['question']}
        ],
        'answer': extract_hash_answer(x['answer'])
    }) # type: ignore
    return data # type: ignore

dataset = get_gsm8k_questions()

# Reward functions
def correctness_reward_func(prompts, completions, answer, **kwargs) -> list[float]:
    responses = [completion[0]['content'] for completion in completions]
    q = prompts[0][-1]['content']
    extracted_responses = [extract_xml_answer(r) for r in responses]
    print('-'*20, f"Question:\n{q}", f"\nAnswer:\n{answer[0]}", f"\nResponse:\n{responses[0]}", f"\nExtracted:\n{extracted_responses[0]}")
    return [2.0 if r == a else 0.0 for r, a in zip(extracted_responses, answer)]

def int_reward_func(completions, **kwargs) -> list[float]:
    responses = [completion[0]['content'] for completion in completions]
    extracted_responses = [extract_xml_answer(r) for r in responses]
    return [0.5 if r.isdigit() else 0.0 for r in extracted_responses]

def strict_format_reward_func(completions, **kwargs) -> list[float]:
    """Reward function that checks if the completion has a specific format."""
    pattern = r"^<reasoning>\n.*?\n</reasoning>\n<answer>\n.*?\n</answer>\n$"
    responses = [completion[0]["content"] for completion in completions]
    matches = [re.match(pattern, r) for r in responses]
    return [0.5 if match else 0.0 for match in matches]

def soft_format_reward_func(completions, **kwargs) -> list[float]:
    """Reward function that checks if the completion has a specific format."""
    pattern = r"<reasoning>.*?</reasoning>\s*<answer>.*?</answer>"
    responses = [completion[0]["content"] for completion in completions]
    matches = [re.match(pattern, r) for r in responses]
    return [0.5 if match else 0.0 for match in matches]

def count_xml(text) -> float:
    count = 0.0
    if text.count("<reasoning>\n") == 1:
        count += 0.125
    if text.count("\n</reasoning>\n") == 1:
        count += 0.125
    if text.count("\n<answer>\n") == 1:
        count += 0.125
        count -= len(text.split("\n</answer>\n")[-1])*0.001
    if text.count("\n</answer>") == 1:
        count += 0.125
        count -= (len(text.split("\n</answer>")[-1]) - 1)*0.001
    return count

def xmlcount_reward_func(completions, **kwargs) -> list[float]:
    contents = [completion[0]["content"] for completion in completions]
    return [count_xml(c) for c in contents]

README.md:   0%|          | 0.00/7.94k [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/2.31M [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/419k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/7473 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/1319 [00:00<?, ? examples/s]

Map:   0%|          | 0/7473 [00:00<?, ? examples/s]

<a name="Train"></a>
### Train the model

Now set up GRPO Trainer and all configurations!

In [7]:
from trl import GRPOConfig, GRPOTrainer

training_args = GRPOConfig(
    use_vllm = True, # use vLLM for fast inference!
    learning_rate = 5e-6,
    adam_beta1 = 0.9,
    adam_beta2 = 0.99,
    weight_decay = 0.1,
    warmup_ratio = 0.1,
    lr_scheduler_type = "cosine",
    # optim = "paged_adamw_8bit",
    optim = "adamw_8bit",
    logging_steps = 1,
    bf16 = is_bfloat16_supported(),
    fp16 = not is_bfloat16_supported(),
    per_device_train_batch_size = 1,
    gradient_accumulation_steps = 1,
    num_generations = 8,
    max_prompt_length = 256,
    # max_completion_length = 200,
    max_completion_length = 300,
    # num_train_epochs = 1,
    # max_steps = 700,
    max_steps = 1250,
    save_steps = 50,
    max_grad_norm = 0.1,
    report_to = "none",
    output_dir = "outputs",
)

And let's run the trainer! If you scroll up, you'll see a table of rewards. The goal is to see the `reward` column increase!

You might have to wait 150 to 200 steps for any action. You'll probably get 0 reward for the first 100 steps. Please be patient!

| Step | Training Loss | reward    | reward_std | completion_length | kl       |
|------|---------------|-----------|------------|-------------------|----------|
| 1    | 0.000000      | 0.125000  | 0.000000   | 200.000000        | 0.000000 |
| 2    | 0.000000      | 0.072375  | 0.248112   | 200.000000        | 0.000000 |
| 3    | 0.000000      | -0.079000 | 0.163776   | 182.500000        | 0.000005 |


In [8]:
trainer = GRPOTrainer(
    model = model,
    processing_class = tokenizer,
    reward_funcs = [
        xmlcount_reward_func,
        soft_format_reward_func,
        strict_format_reward_func,
        int_reward_func,
        correctness_reward_func,
    ],
    args = training_args,
    train_dataset = dataset,
)
trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs = 1
   \\   /|    Num examples = 7,473 | Num Epochs = 1
O^O/ \_/ \    Batch size per device = 1 | Gradient Accumulation steps = 1
\        /    Total batch size = 1 | Total steps = 1,250
 "-____-"     Number of trainable parameters = 369,623,040


-------------------- Question:
A concert ticket costs $40. Mr. Benson bought 12 tickets and received a 5% discount for every ticket bought that exceeds 10. How much did Mr. Benson pay in all? 
Answer:
476 
Response:
<reasoning>
1. Calculate the cost of the first 10 tickets without discount: 10 tickets * $40/ticket = $400
2. Calculate the discount for the remaining 2 tickets: 5% of $40 = $40 * 0.05 = $2
3. Calculate the cost of the remaining 2 tickets with discount: $40 - $2 = $38 per ticket
4. Calculate the total cost for the remaining 2 tickets: $38 * 2 = $76
5. Add the cost of the first 10 tickets and the remaining 2 tickets to get the total cost: $400 + $76 = $476
</reasoning>
<answer>
Mr. Benson paid $476 in all.
</answer> 
Extracted:
Mr. Benson paid $476 in all.


Step,Training Loss,reward,reward_std,completion_length,kl
1,0.000000,-0.111250,0.064694,198.625000,0.000323
2,0.000000,-0.084250,0.204563,237.250000,0.000363
3,0.000000,-0.378875,0.136779,264.625000,0.000269
4,0.000000,0.246375,0.941833,179.125000,0.000343
5,0.000000,-0.053000,0.067358,186.250000,0.000289
6,0.000000,-0.147625,0.135638,251.125000,0.000334
7,0.000000,-0.184250,0.088235,232.625000,0.000264
8,0.000000,-0.013250,0.189535,150.250000,0.000610
9,0.000000,-0.186000,0.148505,198.750000,0.000385
10,0.000000,-0.227375,0.317860,289.625000,0.000284


Streaming output truncated to the last 5000 lines.
Answer:
320 
Response:
<reasoning>To calculate Jenna's earnings for a round trip, first determine the total miles she drives in a round trip: 400 ( one way) x 2 = 800 miles. Next, multiply the total miles driven by the rate per mile: 800 miles x $0.40 per mile. Please note that the rate should be used as a decimal: $0.40 = 0.40.</reasoning>

<answer>80 * 4 = 320</answer> 
Extracted:
80 * 4 = 320
-------------------- Question:
Chip has a $50.00 balance on his credit card.  Since he didn't pay it off, he will be charged a 20% interest fee.  He puts $20.00 on his credit card the following month and doesn't make any payments to his debt.  He is hit with another 20% interest fee.  What's the current balance on his credit card? 
Answer:
96 
Response:
<reasoning>First, let's calculate the interest for the first month. The interest for the first month is $50.00 * 20% = $10.00. This adds up to the initial balance making Chip's new balance $50.0

TrainOutput(global_step=1250, training_loss=0.009419155371462694, metrics={'train_runtime': 29551.8734, 'train_samples_per_second': 0.042, 'train_steps_per_second': 0.042, 'total_flos': 0.0, 'train_loss': 0.009419155371462694})

<a name="Inference"></a>
### Inference
Now let's try the model we just trained! First, let's first try the model without any GRPO trained:

In [9]:
import textwrap

text = tokenizer.apply_chat_template([
    {"role" : "user", "content" : "Calculate pi."},
], tokenize = False, add_generation_prompt = True)

from vllm import SamplingParams
sampling_params = SamplingParams(
    temperature = 0.8,
    top_p = 0.95,
    max_tokens = 1024,
)
output = model.fast_generate(
    [text],
    sampling_params = sampling_params,
    lora_request = None,
)[0].outputs[0].text

print()
print(textwrap.fill(output, width=80, replace_whitespace=False))
output

Processed prompts: 100%|██████████| 1/1 [00:21<00:00, 21.08s/it, est. speed input: 8.59 toks/s, output: 32.26 toks/s]


Calculating the value of pi (π) can be done using various methods, each with
different levels of complexity and precision. One of the simplest methods is the
Monte Carlo method, but it's not very precise. A more precise method is the
Gauss-Legendre algorithm, but it's also more complex. Here are a couple of
simple methods:

### 1. Using the Leibniz Formula
The Leibniz formula for π is:
\[ \pi = 4 \left( 1 - \frac{1}{3} + \frac{1}{5} - \frac{1}{7} + \frac{1}{9} -
\cdots \right) \]

This series converges very slowly, but it's straightforward
to implement.

### 2. Using the Machin-like Formula
A more efficient series for
π is the Machin-like formula:
\[ \pi = 16 \arctan \left( \frac{1}{5} \right) - 4
\arctan \left( \frac{1}{239} \right) \]

This can be computed using the Taylor
series for arctan.

### 3. Using the Bailey–Borwein–Plouffe (BBP) Formula
The
BBP formula is:
\[ \pi = \sum_{k=0}^{\infty} \left( \frac{1}{16^k} \left(
\frac{4}{8k+1} - \frac{2}{8k+4} - \frac{1}{8k+5} - \frac{1}{8

'Calculating the value of pi (π) can be done using various methods, each with different levels of complexity and precision. One of the simplest methods is the Monte Carlo method, but it\'s not very precise. A more precise method is the Gauss-Legendre algorithm, but it\'s also more complex. Here are a couple of simple methods:\n\n### 1. Using the Leibniz Formula\nThe Leibniz formula for π is:\n\\[ \\pi = 4 \\left( 1 - \\frac{1}{3} + \\frac{1}{5} - \\frac{1}{7} + \\frac{1}{9} - \\cdots \\right) \\]\n\nThis series converges very slowly, but it\'s straightforward to implement.\n\n### 2. Using the Machin-like Formula\nA more efficient series for π is the Machin-like formula:\n\\[ \\pi = 16 \\arctan \\left( \\frac{1}{5} \\right) - 4 \\arctan \\left( \\frac{1}{239} \\right) \\]\n\nThis can be computed using the Taylor series for arctan.\n\n### 3. Using the Bailey–Borwein–Plouffe (BBP) Formula\nThe BBP formula is:\n\\[ \\pi = \\sum_{k=0}^{\\infty} \\left( \\frac{1}{16^k} \\left( \\frac{4}{8k+1

And now with the LoRA we just trained with GRPO - we first save the LoRA first!

In [10]:
model.save_lora("grpo_saved_lora")

Now we load the LoRA and test:

In [11]:
text = tokenizer.apply_chat_template([
    {"role" : "system", "content" : SYSTEM_PROMPT},
    {"role" : "user", "content" : "Calculate pi."},
], tokenize = False, add_generation_prompt = True)

from vllm import SamplingParams
sampling_params = SamplingParams(
    temperature = 0.8,
    top_p = 0.95,
    max_tokens = 1024,
)
output = model.fast_generate(
    text,
    sampling_params = sampling_params,
    lora_request = model.load_lora("grpo_saved_lora"),
)[0].outputs[0].text

print()
print(textwrap.fill(output, width=80, replace_whitespace=False))
output

Processed prompts: 100%|██████████| 1/1 [00:05<00:00,  5.36s/it, est. speed input: 6.15 toks/s, output: 27.04 toks/s]


<reasoning>π (pi) is an irrational number, meaning its decimal representation
never ends and never repeats. It is commonly approximated to 3.14159, but for
most practical purposes, it is simplified to 3.14, while in other cases, it is
represented by the symbol 'π'. There is no way to calculate the exact value of
pi, it is not a calculated value but rather a mathematical constant that
represents the ratio of a circle's circumference to its diameter. However, we
can use formulas like the Monte Carlo method, Buffon's needle problem, or series
methods to approximate the value of π.</reasoning>
<answer>3.14</answer>


"<reasoning>π (pi) is an irrational number, meaning its decimal representation never ends and never repeats. It is commonly approximated to 3.14159, but for most practical purposes, it is simplified to 3.14, while in other cases, it is represented by the symbol 'π'. There is no way to calculate the exact value of pi, it is not a calculated value but rather a mathematical constant that represents the ratio of a circle's circumference to its diameter. However, we can use formulas like the Monte Carlo method, Buffon's needle problem, or series methods to approximate the value of π.</reasoning>\n<answer>3.14</answer>"

Our reasoning model is much better - it's not always correct, since we only trained it for an hour or so - it'll be better if we extend the sequence length and train for longer!

<a name="Save"></a>
### Saving to float16 for VLLM

We also support saving to `float16` directly. Select `merged_16bit` for float16 or `merged_4bit` for int4. We also allow `lora` adapters as a fallback. Use `push_to_hub_merged` to upload to your Hugging Face account! You can go to https://huggingface.co/settings/tokens for your personal tokens.

In [12]:
# Merge to 16bit
if False: model.save_pretrained_merged("model", tokenizer, save_method = "merged_16bit",)
if False: model.push_to_hub_merged(hf_hub_path, tokenizer, save_method = "merged_16bit", token = "")

# Just LoRA adapters
if False: model.save_pretrained_merged("model", tokenizer, save_method = "lora",)
if False: model.push_to_hub_merged(hf_hub_path, tokenizer, save_method = "lora", token = "")

# Merge to 4bit
if False: model.save_pretrained_merged("model", tokenizer, save_method = "merged_4bit",)
if False: model.push_to_hub_merged(hf_hub_path, tokenizer, save_method = "merged_4bit", token = "")

### GGUF / llama.cpp Conversion
To save to `GGUF` / `llama.cpp`, we support it natively now! We clone `llama.cpp` and we default save it to `q8_0`. We allow all methods like `q4_k_m`. Use `save_pretrained_gguf` for local saving and `push_to_hub_gguf` for uploading to HF.

Some supported quant methods (full list on our [Wiki page](https://github.com/unslothai/unsloth/wiki#gguf-quantization-options)):
* `q8_0` - Fast conversion. High resource use, but generally acceptable.
* `q4_k_m` - Recommended. Uses Q6_K for half of the attention.wv and feed_forward.w2 tensors, else Q4_K.
* `q5_k_m` - Recommended. Uses Q6_K for half of the attention.wv and feed_forward.w2 tensors, else Q5_K.

[**NEW**] To finetune and auto export to Ollama, try our [Ollama notebook](https://colab.research.google.com/drive/1WZDi7APtQ9VsvOrQSSC5DDtxq159j8iZ?usp=sharing)

In [16]:
# Save to 8bit Q8_0
if False: model.save_pretrained_gguf("model", tokenizer,)
# Remember to go to https://huggingface.co/settings/tokens for a token!
# And change hf to your username!
if False: model.push_to_hub_gguf(hf_hub_path, tokenizer)

# Save to 16bit GGUF
if False: model.save_pretrained_gguf("model", tokenizer, quantization_method = "f16")
if True: model.push_to_hub_gguf(hf_hub_path, tokenizer, quantization_method = "f16")

# Save to q4_k_m GGUF
if False: model.save_pretrained_gguf("model", tokenizer, quantization_method = "q4_k_m")
if False: model.push_to_hub_gguf(hf_hub_path, tokenizer, quantization_method = "q4_k_m")

# Save to multiple GGUF options - much faster if you want multiple!
if False:
    model.push_to_hub_gguf(
        hf_hub_path, # Change hf to your username!
        tokenizer,
        quantization_method = ["q4_k_m", "q8_0", "q5_k_m",],
    )

Unsloth: Merging 4bit and LoRA weights to 16bit...
Unsloth: Will use up to 45.07 out of 83.48 RAM for saving.
Unsloth: Saving model... This might take 5 minutes ...


100%|██████████| 40/40 [03:23<00:00,  5.08s/it]


Unsloth: Saving tokenizer... Done.
Done.
==((====))==  Unsloth: Conversion from QLoRA to GGUF information
   \\   /|    [0] Installing llama.cpp might take 3 minutes.
O^O/ \_/ \    [1] Converting HF to GGUF 16bits might take 3 minutes.
\        /    [2] Converting GGUF 16bits to ['f16'] might take 10 minutes each.
 "-____-"     In total, you will have to wait at least 16 minutes.

Unsloth: Installing llama.cpp. This might take 3 minutes...
Unsloth: [1] Converting model at jimwhite/mistral-grpo_20250215-035142 into f16 GGUF format.
The output location will be /content/drive/MyDrive/Data/Unsloth/mistral-grpo_20250215-035142/jimwhite/mistral-grpo_20250215-035142/unsloth.F16.gguf
This might take 3 minutes...
INFO:hf-to-gguf:Loading model: mistral-grpo_20250215-035142
INFO:gguf.gguf_writer:gguf: This GGUF file is for Little Endian only
INFO:hf-to-gguf:Exporting model...
INFO:hf-to-gguf:gguf: loading model weight map from 'model.safetensors.index.json'
INFO:hf-to-gguf:gguf: loading model par

  0%|          | 0/1 [00:00<?, ?it/s]

unsloth.F16.gguf:   0%|          | 0.00/47.2G [00:00<?, ?B/s]

Saved GGUF to https://huggingface.co/jimwhite/mistral-grpo_20250215-035142


JPW: Looks like default_system_message is still the base model's which surely isn't correct and should be the system message used for QRPO.

Now, use the `model-unsloth.gguf` file or `model-unsloth-Q4_K_M.gguf` file in llama.cpp or a UI based system like Jan or Open WebUI. You can install Jan [here](https://github.com/janhq/jan) and Open WebUI [here](https://github.com/open-webui/open-webui)

And we're done! If you have any questions on Unsloth, we have a [Discord](https://discord.gg/unsloth) channel! If you find any bugs or want to keep updated with the latest LLM stuff, or need help, join projects etc, feel free to join our Discord!

Some other links:
1. Llama 3.2 Conversational notebook. [Free Colab](https://colab.research.google.com/github/unslothai/notebooks/blob/main/nb/Llama3.2_(1B_and_3B)-Conversational.ipynb)
2. Saving finetunes to Ollama. [Free notebook](https://colab.research.google.com/github/unslothai/notebooks/blob/main/nb/Llama3_(8B)-Ollama.ipynb)
3. Llama 3.2 Vision finetuning - Radiography use case. [Free Colab](https://colab.research.google.com/github/unslothai/notebooks/blob/main/nb/Llama3.2_(11B)-Vision.ipynb)
6. See notebooks for DPO, ORPO, Continued pretraining, conversational finetuning and more on our [documentation](https://docs.unsloth.ai/get-started/unsloth-notebooks)!

<div class="align-center">
  <a href="https://unsloth.ai"><img src="https://github.com/unslothai/unsloth/raw/main/images/unsloth%20new%20logo.png" width="115"></a>
  <a href="https://discord.gg/unsloth"><img src="https://github.com/unslothai/unsloth/raw/main/images/Discord.png" width="145"></a>
  <a href="https://docs.unsloth.ai/"><img src="https://github.com/unslothai/unsloth/blob/main/images/documentation%20green%20button.png?raw=true" width="125"></a>

  Join Discord if you need help + ⭐️ <i>Star us on <a href="https://github.com/unslothai/unsloth">Github</a> </i> ⭐️
</div>
